In [1]:
import os 
import datetime
from pathlib import Path
from dotenv import load_dotenv, find_dotenv

basepath = Path(os.getcwd())
# make sure your working directory is the repository root.
if basepath.name != "idp-radio-1":
    os.chdir(basepath.parent.parent.parent)
load_dotenv(find_dotenv())

%load_ext autoreload
%autoreload 2
os.getcwd()

'/srv/idp-radio-1'

In [2]:
import os 
import tensorflow as tf
from pathlib import Path

In [3]:
# Run this before loading other dependencies, otherwise they might occupy memory on gpu 0 by default and it will stay that way

# Specify which GPU(s) to use
os.environ["CUDA_VISIBLE_DEVICES"] = "1"  # Or 2, 3, etc. other than 0

config = tf.compat.v1.ConfigProto(allow_soft_placement=True, log_device_placement=True)
config.gpu_options.allow_growth = True
tf.compat.v1.Session(config=config)


Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device
/job:localhost/replica:0/task:0/device:XLA_GPU:0 -> device: XLA_GPU device
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: GeForce RTX 2080 Ti, pci bus id: 0000:42:00.0, compute capability: 7.5



In [4]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.densenet import DenseNet121
from tensorflow.keras.applications.resnet_v2 import ResNet101V2
from src.architectures.simple.simple_base import SimpleBaseArchitecture
from src.architectures.adv.guendel19 import densenet

from src.architectures.benchmarks.benchmark import Benchmark, Experiment
from src.architectures.benchmarks.benchmark_definitions import Chexpert_Benchmark, Chestxray14_Benchmark, simple_architecture_experiment, generate_benchmarks, METRICS, CHEXPERT_COLUMNS, CHESTXRAY14_COLUMNS
from src.metrics.metrics import F2Score
from src.metrics.losses import WeightedBinaryCrossentropy, compute_class_weight

In [5]:
#CHEXPERT_BENCHMARKS, CHESTXRAY14_BENCHMARKS = generate_benchmarks ()

In [ ]:
#CHEXPERT_BENCHMARKS.keys() 

In [ ]:
#CHESTXRAY14_BENCHMARKS.keys()

In [ ]:
#[m.name for m in METRICS]

In [ ]:
#CHEXPERT_COLUMNS

In [ ]:
!remote_access/get_tunnels.sh

In [6]:
reduced_columns = ['Enlarged Cardiomediastinum',
                    'Cardiomegaly',
                    'Lung Opacity',
                    'Lung Lesion',
                    'Edema',
                    'Consolidation',
                    'Pneumonia',
                    'Atelectasis',
                    'Pneumothorax',
                    'Pleural Effusion',
                    'Pleural Other',
                    'Fracture']
len(reduced_columns)

12

In [7]:
chexpert_benchmarks, _ = generate_benchmarks(path = Path(os.environ.get("CHEXPERT_FULL_PREPROCESSED_DATASET_DIRECTORY")),
                                                 classes=reduced_columns,
                                                 #train_labels = "nofinding_train.csv",
                                                 batch_sizes = {"b": 32},
                                                 epoch_sizes = {"e": 12},
                                                 crop = {"C1": False},
                                                 nan_replacement = float("NaN"),
                                                 split_seed = 6122156, )

Chestxray_BCE_E12_B32_C1_N12 could not be created
Chestxray_WBCE_E12_B32_C1_N12 could not be created
Chestxray_CWBCE_E12_B32_C1_N12 could not be created


In [ ]:
chexpert_exp = simple_architecture_experiment(chexpert_benchmarks["CWBCE_E12_B32_C1_N12"], DenseNet121, reduced_columns)

In [8]:
mymodel = SimpleBaseArchitecture(DenseNet121, len(reduced_columns))

In [10]:
optim = chexpert_benchmarks["CWBCE_E12_B32_C1_N12"].optimizer
optim

In [11]:
loss = chexpert_benchmarks["CWBCE_E12_B32_C1_N12"].loss
loss

In [12]:
metrics = chexpert_benchmarks["CWBCE_E12_B32_C1_N12"].metrics
metrics

In [13]:
mymodel.compile(optimizer=optim,
               loss=loss,
               metrics=metrics)

In [ ]:
train_result = mymodel.fit(x=traingen,
                           steps_per_epoch=len(traingen),
                           epochs=2,)

In [14]:
traingen = chexpert_benchmarks["BCE_E12_B32_C1_N12"].traingen
first_batch, ytrue = traingen[0]
ytrue

array([[nan, nan,  1., nan, nan, nan, nan, nan,  0.,  1., nan, nan],
       [nan,  1.,  1., nan,  0., nan, nan, nan, nan,  1., nan, nan],
       [nan, nan,  1., nan, nan, nan, nan,  1., nan,  1., nan, nan],
       [nan, nan,  1., nan, nan, nan, nan, nan, nan,  0.,  0., nan],
       [nan,  1.,  0., nan, nan,  0., nan,  0., nan,  1., nan, nan],
       [ 0., nan,  0., nan,  0., nan, nan, nan, nan,  0., nan, nan],
       [nan,  1.,  0., nan, nan,  1., nan,  0.,  0., nan, nan,  0.],
       [nan, nan,  1., nan, nan,  0., nan,  0., nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan, nan,  1.,  0., nan, nan, nan],
       [nan,  1.,  1., nan, nan, nan, nan, nan, nan,  1., nan, nan],
       [nan, nan,  1.,  1., nan, nan, nan, nan,  0.,  0., nan, nan],
       [nan, nan, nan, nan,  1.,  1., nan, nan, nan, nan, nan, nan],
       [ 0., nan,  1., nan, nan, nan, nan, nan, nan,  1., nan, nan],
       [nan, nan,  1., nan,  1., nan, nan, nan,  0.,  1., nan, nan],
       [nan, nan, nan, nan, nan, n

In [19]:
mymodel.predict_on_batch(first_batch)

array([[5.00154644e-02, 2.03672081e-01, 9.72444296e-01, 9.65218306e-01,
        8.58597457e-01, 2.55264696e-02, 9.96382833e-01, 8.19808993e-05,
        1.34775907e-01, 9.84337151e-01, 9.88545597e-01, 7.58847222e-04],
       [3.10931890e-03, 9.12559256e-02, 9.74691689e-01, 9.26402509e-01,
        8.47091138e-01, 4.27976474e-02, 9.97799098e-01, 6.27276449e-06,
        2.02304974e-01, 9.83514428e-01, 9.79969919e-01, 2.89483345e-04],
       [3.30315940e-02, 9.97245014e-02, 9.94085670e-01, 5.66407263e-01,
        8.15448701e-01, 3.94033827e-02, 9.87719297e-01, 2.44019539e-05,
        2.52821714e-01, 9.17831898e-01, 9.36409354e-01, 1.60817010e-03],
       [6.55725040e-03, 7.21208528e-02, 9.39567924e-01, 9.14177418e-01,
        8.01517487e-01, 6.69262186e-02, 9.98201489e-01, 2.21702605e-04,
        5.49355447e-01, 9.88143861e-01, 9.40692544e-01, 1.32368412e-03],
       [4.18927614e-03, 2.18521968e-01, 9.82300341e-01, 9.52544630e-01,
        9.61109161e-01, 7.78625086e-02, 9.96241689e-01, 7.68

In [26]:
mymodel.train_step()

AttributeError: 'tuple' object has no attribute 'rank'

In [27]:
next(iter(traingen))

(array([[[[ 20,  20,  20],
          [ 20,  20,  20],
          [ 20,  20,  20],
          ...,
          [ 14,  14,  14],
          [ 14,  14,  14],
          [ 14,  14,  14]],
 
         [[ 21,  21,  21],
          [ 21,  21,  21],
          [ 21,  21,  21],
          ...,
          [ 13,  13,  13],
          [ 13,  13,  13],
          [ 13,  13,  13]],
 
         [[ 21,  21,  21],
          [ 21,  21,  21],
          [ 21,  21,  21],
          ...,
          [ 12,  12,  12],
          [ 12,  12,  12],
          [ 12,  12,  12]],
 
         ...,
 
         [[207, 207, 207],
          [208, 208, 208],
          [211, 211, 211],
          ...,
          [119, 119, 119],
          [107, 107, 107],
          [ 98,  98,  98]],
 
         [[204, 204, 204],
          [213, 213, 213],
          [219, 219, 219],
          ...,
          [111, 111, 111],
          [ 95,  95,  95],
          [ 88,  88,  88]],
 
         [[204, 204, 204],
          [218, 218, 218],
          [225, 225, 225],
   

In [ ]:
predictions = experiment_inceptionnet_chexpert.model.predict(
            experiment_inceptionnet_chexpert.benchmark.testgen, steps=len(experiment_inceptionnet_chexpert.benchmark.testgen), verbose=1)

In [ ]:
true_labels = experiment_inceptionnet_chexpert.benchmark.testgen.get_encoded_labels()

In [ ]:
# model_guendel_chestxray14 = densenet(classes=len(CHESTXRAY14_COLUMNS))
model_guendel_chexpert = densenet(classes=len(CHEXPERT_COLUMNS))

#experiment_guendel_chestxray14 = Experiment(chestxray14_benchmark, model_guendel_chestxray14)
experiment_guendel_chexpert = Experiment(CHEXPERT_BENCHMARKS["WBCE_E10_B32"], model_guendel_chexpert, model_name="test_WBCE_32")



In [ ]:
#experiment_guendel_chestxray14_result =  experiment_guendel_chestxray14.run()

In [ ]:
experiment_guendel_chexpert_result =  experiment_guendel_chexpert.run()

In [ ]:
experiment_guendel_chexpert.evaluate()

In [ ]:
experiment_guendel_chexpert.save(upload=False)

In [ ]:
print("done")

In [ ]:
CHEXPERT_BENCHMARKS["WBCE_E10_B32"].as_dict()

In [ ]:
y_true

In [ ]:
weights_pos = tf.ones((4)) * 20.0
weights_neg = tf.ones((4)) * 0.5
y_true = tf.constant([[0, 1, float("nan"), 1], [float("nan"), 0, 1, float("nan")]], dtype=tf.float32)
y_pred = tf.constant([[0.1, 0.2, 0.3, 0.4], [0.5, 0.6, 0.7, 0.8]])
loss = tf.keras.losses.binary_crossentropy(y_true, y_pred)
loss

In [ ]:
mask = tf.math.is_nan(y_true)
mask

In [ ]:
tf.boolean_mask(y_pred, mask)

In [ ]:
from tensorflow.keras import backend as K
from tensorflow.python.ops.losses import util as tf_losses_util
from tensorflow.python.framework import tensor_util
from tensorflow.python.framework import constant_op
from tensorflow.python.ops import math_ops
from tensorflow.python.ops import clip_ops
from tensorflow.python.keras import backend_config

In [ ]:
epsilon = backend_config.epsilon
if tensor_util.is_tensor(y_pred) and tensor_util.is_tensor(y_true):
    y_pred, y_true = tf_losses_util.squeeze_or_expand_dimensions(
        y_pred, y_true)

epsilon_ = constant_op.constant(
    epsilon(), dtype=y_pred.dtype.base_dtype)
y_pred = clip_ops.clip_by_value(y_pred, epsilon_, 1. - epsilon_)

# Compute cross entropy from probabilities.
bce_pos = y_true * math_ops.log(y_pred + epsilon_)
bce_neg = (1 - y_true) * math_ops.log(1 - y_pred + epsilon_)
bce = bce_pos * weights_pos +  bce_neg * weights_neg
bce

In [ ]:
tf.reduce_mean(tf.boolean_mask(-bce, tf.math.is_finite(bce)))

In [ ]:
y_true * math_ops.log(y_pred + epsilon_)

In [ ]:
tf.boolean_mask(-bce, tf.math.is_finite(bce))

In [ ]:
bench = Chexpert_Benchmark("Chexpert_CWBCE",
                           epochs=12,
                           batch_size=1,
                           metrics=METRICS,
                           crop=True, 
                           nan_replacement = float("nan"))
bench.loss = WeightedBinaryCrossentropy(bench.positive_weights,
                       bench.negative_weights)

In [ ]:
bench.traingen[0]